In [5]:
import psycopg2

In [ ]:
def create_connection():
    try:
        # Paramètres de connexion
        connection = psycopg2.connect(
            database="Reviews",
            user="postgres",
            password="2003",
            host="localhost",
            port="5432"
        )
        
        # Création d'un curseur
        cursor = connection.cursor()
        
        # Afficher la version de PostgreSQL
        cursor.execute("SELECT version();")
        version = cursor.fetchone()
        print("Connected to PostgreSQL. Version:", version)
        
        return connection, cursor
        
    except (Exception, Error) as error:
        print("Erreur lors de la connexion à PostgreSQL:", error)
        return None, None

def close_connection(connection, cursor):
    if cursor:
        cursor.close()
    if connection:
        connection.close()
        print("La connexion à PostgreSQL est fermée")


In [14]:
def create_bank_reviews_table(connection, cursor):
    try:
        # Création de la table
        create_table_query = """
        CREATE TABLE IF NOT EXISTS bank_reviews (
            id SERIAL PRIMARY KEY,
            bank_name VARCHAR(100),
            branch_name VARCHAR(200),
            location VARCHAR(200),
            review TEXT,
            rating DECIMAL(2,1),
            review_date TIMESTAMP,
            sentiment VARCHAR(20),
            Topic VARCHAR(100),
            CONSTRAINT rating_check CHECK (rating >= 0 AND rating <= 5)
        );
        """
        
        cursor.execute(create_table_query)
        connection.commit()
        print("Table bank_reviews créée avec succès!")
        
    except Exception as e:
        print(f"Erreur lors de la création de la table : {e}")
        connection.rollback()

# Utilisation
conn, cur = create_connection()
if conn is not None:
    create_bank_reviews_table(conn, cur)
    close_connection(conn, cur)

Connected to PostgreSQL. Version: ('PostgreSQL 14.17 (Homebrew) on x86_64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit',)
Table bank_reviews créée avec succès!
La connexion à PostgreSQL est fermée


In [ ]:
"""""
# Insérer des données
def insert_data(cursor, name, value):
    cursor.execute(
        "INSERT INTO test_table (name, value) VALUES (%s, %s)",
        (name, value)
    )

# Lire des données
def read_data(cursor):
    cursor.execute("SELECT * FROM test_table")
    return cursor.fetchall()

# Mettre à jour des données
def update_data(cursor, id, new_value):
    cursor.execute(
        "UPDATE test_table SET value = %s WHERE id = %s",
        (new_value, id)
    )

# Supprimer des données
def delete_data(cursor, id):
    cursor.execute(
        "DELETE FROM test_table WHERE id = %s",
        (id,)
    )
"""


In [15]:
import pandas as pd
from datetime import datetime

def insert_reviews_data(connection, cursor, csv_file):
    try:
        # Lire le fichier CSV
        df = pd.read_csv(csv_file)
        
        # Préparer la requête d'insertion
        insert_query = """
        INSERT INTO bank_reviews 
        (bank_name, branch_name, location, review, rating, review_date, sentiment, Topic)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        # Insérer chaque ligne du DataFrame
        for index, row in df.iterrows():
            try:
                # Convertir la date en format datetime avec gestion des microsecondes variables
                try:
                    # Essayer d'abord le format standard
                    date = datetime.strptime(row['date'], '%Y-%m-%dT%H:%M:%S.%fZ')
                except ValueError:
                    # Si échec, tronquer les microsecondes à 6 chiffres
                    date_str = row['date']
                    if '.' in date_str:
                        base, ms = date_str.split('.')
                        ms = ms.replace('Z', '')[:6]  # Garder seulement 6 chiffres pour les microsecondes
                        date_str = f"{base}.{ms}Z"
                    date = datetime.strptime(date_str, '%Y-%m-%dT%H:%M:%S.%fZ')
                
                # Préparer les valeurs à insérer
                values = (
                    row['bank_name'],
                    row['branch_name'],
                    row['location'],
                    row['review'],
                    float(row['rating']),
                    date,
                    row['sentiment_label'],
                    row['dominant_topic']
                )
                
                # Exécuter l'insertion
                cursor.execute(insert_query, values)
                
            except Exception as e:
                print(f"Erreur lors de l'insertion de la ligne {index}: {e}")
                continue
        
        # Valider les insertions
        connection.commit()
        print("Données insérées avec succès!")
        
    except Exception as e:
        print(f"Erreur lors de l'insertion des données: {e}")
        connection.rollback()
# Utilisation de la fonction
try:
    conn, cur = create_connection()
    if conn is not None:
        csv_file = 'bank_reviews_analysis.csv'
        insert_reviews_data(conn, cur, csv_file)
        
        # Vérifier le nombre d'entrées
        cur.execute("SELECT COUNT(*) FROM bank_reviews")
        count = cur.fetchone()[0]
        print(f"Nombre total d'entrées dans la table: {count}")
        
        close_connection(conn, cur)
except Exception as e:
    print(f"Erreur: {e}")

Connected to PostgreSQL. Version: ('PostgreSQL 14.17 (Homebrew) on x86_64-apple-darwin23.6.0, compiled by Apple clang version 16.0.0 (clang-1600.0.26.6), 64-bit',)
Données insérées avec succès!
Nombre total d'entrées dans la table: 3783
La connexion à PostgreSQL est fermée
